In [1]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import yaml

ССылки на файлы

In [2]:
ASU_dataset = Path('data/test_dataset_test.csv')

Загрузка баового датасета

In [3]:

asu_frame = pd.read_csv(ASU_dataset, encoding='')
asu_frame.head(2)

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Город_ПП,Общежитие,Наличие_Матери,Наличие_Отца,Страна_Родители,Опекунство,Село,Иностранец,КодФакультета,СрБаллАттестата
0,64996,20608,2014,Жен,ДН,Английский язык,1995-11-03 00:00:00.000,"МБОУ ""СОШ №55""","Алтайский край, Барнаул г",2014.0,...,Барнаул г,0.0,0,0.0,Россия,0.0,0.0,0.0,40.0,49.0
1,71837,20613,2015,Жен,ДН,Немецкий язык,1995-12-10 00:00:00.000,"МБОУ ""СОШ №1""","Алтайский край, Рубцовск г",2014.0,...,Барнаул г,0.0,0,0.0,Россия,0.0,1.0,0.0,26.0,77.0


Очистка значений датафрейма

In [4]:
asu_frame['male'] = 0
asu_frame['female'] = 0

asu_frame.loc[asu_frame['Пол'] == 'муж', 'Пол'] = 'Муж'
asu_frame.loc[asu_frame['Пол'] == 'жен', 'Пол'] = 'Жен'

asu_frame.loc[asu_frame['Пол'] == 'Муж', 'male'] = 1
asu_frame.loc[asu_frame['Пол'] == 'Жен', 'female'] = 1
asu_frame['sex_summ'] = asu_frame['male'] + asu_frame['female']
asu_frame.loc[asu_frame['sex_summ'] != 1].shape[0]

3

In [5]:
#asu_frame['birth_year'] = asu_frame['Дата_Рождения'].dt.to_period('Y')
asu_frame['Дата_Рождения']
asu_frame['birth_year'] = pd.to_datetime(asu_frame['Дата_Рождения'])
asu_frame['birth_year'].dt.to_period('Y')
asu_frame['birth_year'].all()

asu_frame['entrance'] = pd.to_datetime(asu_frame['Год_Поступления'], format='%Y')
asu_frame['entrance'].sort_values()



3915   2006-01-01
3871   2007-01-01
5156   2007-01-01
1454   2009-01-01
5141   2009-01-01
          ...    
4100   2020-01-01
2328   2020-01-01
2729   2020-01-01
1791   2020-01-01
424    2020-01-01
Name: entrance, Length: 6691, dtype: datetime64[ns]

In [6]:
asu_frame['birth_year_int'] = asu_frame['Дата_Рождения'].str.slice(start=0, stop=4).astype('int64')
asu_frame['birth_year_int']

0       1995
1       1995
2       1973
3       1983
4       1994
        ... 
6686    1992
6687    1995
6688    1996
6689    1996
6690    1992
Name: birth_year_int, Length: 6691, dtype: int64

In [7]:
frame_size = asu_frame.shape
frame_size

(6691, 29)

In [8]:
asu_frame.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Пол', 'Основания',
       'Изучаемый_Язык', 'Дата_Рождения', 'Уч_Заведение', 'Где_Находится_УЗ',
       'Год_Окончания_УЗ', 'Пособие', 'Страна_ПП', 'Регион_ПП', 'Город_ПП',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'male', 'female', 'sex_summ', 'birth_year', 'entrance',
       'birth_year_int'],
      dtype='object')

In [9]:
asu_frame['Статус'] = 0

In [10]:
asu_frame.groupby('Статус')['СрБаллАттестата'].describe()

,count,mean,std,min,25%,50%,75%,max
Статус,,,,,,,,
0,6691.0,71.681369,250.386989,0.0,47.0,61.0,76.0,7232.0


In [11]:
asu_frame.groupby('Статус')['Село'].value_counts()

Статус  Село
0       0.0     5170
        1.0     1484
Name: Село, dtype: int64

In [12]:
asu_frame.pivot_table(index='Пол', columns='Статус')

,ID,birth_year_int,female,male,sex_summ,Год_Окончания_УЗ,Год_Поступления,Иностранец,Код_группы,КодФакультета,Наличие_Матери,Наличие_Отца,Общежитие,Опекунство,Пособие,Село,СрБаллАттестата
Статус,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Пол,,,,,,,,,,,,,,,,,
Жен,69852.612579,1993.969560,1,0,1,2013.759388,2014.925786,0.059759,18300.450566,33.318239,0.499119,0.498868,0.219722,0.000503,0.0,0.238938,72.881648
Муж,70708.725028,1994.086251,0,1,1,2013.861066,2015.088463,0.086499,18327.166605,31.448950,0.504607,0.503502,0.227947,0.001474,0.0,0.199926,69.960004


In [13]:
graduate = asu_frame.loc[asu_frame['Статус'] == 4]['Статус'].count()
expelled = asu_frame.loc[asu_frame['Статус'] == 3]['Статус'].count()
vacation = asu_frame.loc[asu_frame['Статус'] == -1]['Статус'].count()
all_students = frame_size[0]
delta = all_students - graduate - expelled - vacation
f'Всего позиций {all_students} закончили ВУЗ {graduate}, отчислено {expelled}, каникулы {vacation}. Отклонение по сумме {delta}'

'Всего позиций 6691 закончили ВУЗ 0, отчислено 0, каникулы 0. Отклонение по сумме 6691'

In [14]:
graduate_frame = asu_frame.loc[asu_frame['Статус'] == 4]
graduate_frame

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,male,female,sex_summ,birth_year,entrance,birth_year_int,Статус


In [15]:
def doly(x):
    
    gh = x['ID'].count() / frame_size[0]
    print(gh)
    return gh

asu_frame.groupby(['Пол', 'Статус']).apply(doly)

0.5940816021521447
0.40547003437453294


Пол  Статус
Жен  0         0.594082
Муж  0         0.405470
dtype: float64

In [16]:
asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()

/tmp/ipykernel_67599/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()
/tmp/ipykernel_67599/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  asu_frame.loc[asu_frame['СрБаллАттестата'] < 5].groupby(asu_frame['birth_year'].dt.to_period('Y'))['entrance'].describe()
/tmp/ipykernel_67599/3035972624.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of

,count,unique,top,freq,first,last
birth_year,,,,,,
1948,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00
1969,1,1,2013-01-01 00:00:00,1,2013-01-01 00:00:00,2013-01-01 00:00:00
1975,1,1,2014-01-01 00:00:00,1,2014-01-01 00:00:00,2014-01-01 00:00:00
1977,2,2,2015-01-01 00:00:00,1,2013-01-01 00:00:00,2015-01-01 00:00:00
1978,1,1,2013-01-01 00:00:00,1,2013-01-01 00:00:00,2013-01-01 00:00:00
1979,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00
1980,1,1,2013-01-01 00:00:00,1,2013-01-01 00:00:00,2013-01-01 00:00:00
1981,1,1,2014-01-01 00:00:00,1,2014-01-01 00:00:00,2014-01-01 00:00:00
1982,1,1,2015-01-01 00:00:00,1,2015-01-01 00:00:00,2015-01-01 00:00:00


In [17]:
old_year = datetime.datetime.strptime('1986', '%Y')
asu_frame.loc[(asu_frame['birth_year'] < old_year) & (asu_frame['СрБаллАттестата'] > 20)]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,male,female,sex_summ,birth_year,entrance,birth_year_int,Статус
2,86587,21210,2018,Жен,ОО,Английский язык,1973-08-09 00:00:00.000,Алтайский государственный медицинский университет,"Алтайский край, г. Барнаул",1997.0,...,0.0,34.0,74.0,0,1,1,1973-08-09,2018-01-01,1973,0
3,73673,22254,2015,Жен,ОО,Английский язык,1983-12-07 00:00:00.000,"ГОУ ВПО ""Алтайский государственный университет""","Алтайский край, Барнаул г",2006.0,...,0.0,53.0,57.0,0,1,1,1983-12-07,2015-01-01,1983,0
6,77774,18616,2016,Жен,СН,Английский язык,1981-11-30 00:00:00.000,Бийский педагогический государственный универс...,"Россия, Алтайский край, г Бийск",2003.0,...,0.0,26.0,56.0,0,1,1,1981-11-30,2016-01-01,1981,0
20,75272,21670,2016,Жен,БН,Английский язык,1981-03-09 00:00:00.000,Барнаульский государственный педагогический ун...,"Россия, Алтайский край, г Барнаул",2003.0,...,0.0,51.0,94.0,0,1,1,1981-03-09,2016-01-01,1981,0
39,58345,15234,2013,Муж,ОО,Немецкий язык,1984-02-14 00:00:00.000,МОСШ с. Березовка Солонешенского района,с. Березовка Солонешенского района,2001.0,...,0.0,48.0,46.0,1,0,1,1984-02-14,2013-01-01,1984,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6608,86325,21005,2018,Жен,БН,Английский язык,1971-10-14 00:00:00.000,Восточный гуманитарный институт,"Казахстан, Восточно-Казахстанская область, г. ...",2003.0,...,1.0,27.0,73.0,0,1,1,1971-10-14,2018-01-01,1971,0
6614,86326,21005,2018,Муж,ОО,Английский язык,1960-03-15 00:00:00.000,Алтайский государственный университет,"Алтайский край, г. Барнаул",1982.0,...,0.0,27.0,99.0,1,0,1,1960-03-15,2018-01-01,1960,0
6657,70541,17380,2015,Жен,ОО,Немецкий язык,1973-08-01 00:00:00.000,Алтайский государственный университет,"Алтайский край, Барнаул г",1995.0,...,0.0,31.0,85.0,0,1,1,1973-08-01,2015-01-01,1973,0
6673,58310,15233,2013,Муж,ОО,Английский язык,1985-11-25 00:00:00.000,КГОШ,г. Барнаул Алтайский край,2003.0,...,0.0,48.0,43.0,1,0,1,1985-11-25,2013-01-01,1985,0


In [18]:
old_year = datetime.datetime.strptime('2004', "%Y")


In [19]:
asu_frame.groupby('Код_группы')['ID'].count().max()

15

# Очистка первичного фрейма

In [20]:
clean_frame = pd.DataFrame()

In [21]:
clean_frame = asu_frame.copy()

In [22]:
clean_frame.loc[clean_frame['Основания'] == 'ЛН', 'Основания'] = 'ДН'
clean_frame.groupby('Основания')['ID'].count()

Основания
БН     873
ДН     973
ОО    2565
СН    2177
ЦН     103
Name: ID, dtype: int64

Колонка иностранный язык

In [23]:
clean_frame.loc[clean_frame['Изучаемый_Язык'].isnull(), 'Изучаемый_Язык'] = '-'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Англиийский', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Английский)', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Английский, немецкий языки', 'Изучаемый_Язык'] = 'Английский язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Иностранный язык (Немецкий)', 'Изучаемый_Язык'] = 'Немецкий язык'
clean_frame.loc[clean_frame['Изучаемый_Язык'] == 'Русский язык', 'Изучаемый_Язык'] = '-'
clean_frame.groupby('Изучаемый_Язык')['ID'].count()

Изучаемый_Язык
-                    397
Английский язык     5219
Немецкий язык       1047
Французский язык      28
Name: ID, dtype: int64

In [24]:
clean_frame.loc[clean_frame['Пол'].isnull(), 'Пол'] = '-'
clean_frame.groupby('Пол')['ID'].count()

Пол
-         3
Жен    3975
Муж    2713
Name: ID, dtype: int64

In [25]:
clean_frame.loc[clean_frame['Пол'] == '-']

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,male,female,sex_summ,birth_year,entrance,birth_year_int,Статус
2887,73077,16010,2015,-,СН,Английский язык,1993-02-15 00:00:00.000,средняя школа № 1 городского округа Хэган,"Китай, Хэйхэ г",2013.0,...,1.0,51.0,36.0,0,0,0,1993-02-15,2015-01-01,1993,0
3545,73286,17194,2015,-,СН,Английский язык,1995-01-20 00:00:00.000,"СШ ""Пингун""","Китай, Харбин г",2015.0,...,1.0,24.0,38.0,0,0,0,1995-01-20,2015-01-01,1995,0
4326,78542,18499,2016,-,СН,Английский язык,1995-08-23 00:00:00.000,СШ №3 уезда Юйчжоу,"Китай, провинция Хэнань, г Уезд Юйчжоу",2014.0,...,1.0,24.0,40.0,0,0,0,1995-08-23,2016-01-01,1995,0


In [26]:
clean_frame.loc[clean_frame['Уч_Заведение'].isnull(), 'Уч_Заведение'] = '-'
clean_frame.groupby('Уч_Заведение')['ID'].count()

Уч_Заведение
-                                                           578
АГТУ им.И.И.Ползунова                                         1
АГТУ им.Ползунова                                             1
АГУ СПО                                                       1
АКПЛ                                                          1
                                                           ... 
школа Дуньсия                                                 1
школа иностранных языков города Цзинань                       1
школа-гимназия №4 г. Майлуу-Суу                               1
школа-гимназия №4 им. С.М. Кирова                             1
школа-гимназия №5 им. Бекмамата Осмонова гор. Жалал-Абад      1
Name: ID, Length: 2245, dtype: int64

In [27]:
clean_frame.loc[clean_frame['Где_Находится_УЗ'].isnull(), 'Где_Находится_УЗ'] = '-'

In [28]:
clean_frame.loc[clean_frame['Пособие'].isnull(), 'Пособие'] = 0
clean_frame.groupby('Пособие')['ID'].count()


Пособие
0.0    6691
Name: ID, dtype: int64

In [29]:
clean_frame.loc[clean_frame['Страна_ПП'].isnull(), 'Страна_ПП'] = '-'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Афганистан', 'Страна_ПП'] = '-'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Киргизия', 'Страна_ПП'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Кыргызстан', 'Страна_ПП'] = 'Кыргызская Республика'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан ВКО', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан Респ', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Казахстан респ', 'Страна_ПП'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'КАЗАХСТАН', 'Страна_ПП'] = 'Республика Казахстан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан', 'Страна_ПП'] = 'Республика Таджикистан'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Таджикистан Респ', 'Страна_ПП'] = 'Республика Таджикистан'

clean_frame.loc[clean_frame['Страна_ПП'] == 'РОССИЯ', 'Страна_ПП'] = 'Россия'
clean_frame.loc[clean_frame['Страна_ПП'] == 'Росссия', 'Страна_ПП'] = 'Россия'

clean_frame.loc[clean_frame['Страна_ПП'] == 'КИТАЙ', 'Страна_ПП'] = 'Китай'
clean_frame.groupby('Страна_ПП')['ID'].count()

Страна_ПП
-                          161
Китай                       82
Кыргызская Республика       37
Монголия                     1
Республика Казахстан       202
Республика Таджикистан      42
Россия                    6161
Туркменистан                 1
Узбекистан                   4
Name: ID, dtype: int64

In [30]:
clean_frame.loc[clean_frame['Регион_ПП'].isnull(), 'Регион_ПП'] = '-'
clean_frame.groupby('Регион_ПП')['ID'].count()

Регион_ПП
-                      290
Алайский край           18
Алматинская обл          6
Алматинская область      1
Алтай Респ              76
                      ... 
пров. Цзилинь            1
пров. Цзянси             1
пров. Шаньдун            1
провинция Хайнань        8
провинция Цзунци         1
Name: ID, Length: 134, dtype: int64

In [31]:
clean_frame.loc[clean_frame['Город_ПП'].isnull(), 'Город_ПП'] = '-'

In [32]:
clean_frame.loc[clean_frame['Общежитие'].isnull(), 'Общежитие'] = 0
clean_frame.groupby('Общежитие')['ID'].count()

Общежитие
0.0    5204
1.0    1487
Name: ID, dtype: int64

In [33]:
clean_frame.loc[clean_frame['Страна_Родители'].isnull(), 'Страна_Родители'] = '-'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Афганистан', 'Страна_Родители'] = '-'

clean_frame.loc[clean_frame['Страна_Родители'] == 'Кыргызия', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Киргизия', 'Страна_Родители'] = 'Кыргызская Республика'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Кыргызстан', 'Страна_Родители'] = 'Кыргызская Республика'

clean_frame.loc[clean_frame['Страна_Родители'] == 'Казахстан', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Казахстан ВКО', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Казахстан Респ', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Казахстан респ', 'Страна_Родители'] = 'Республика Казахстан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'КАЗАХСТАН', 'Страна_Родители'] = 'Республика Казахстан'

clean_frame.loc[clean_frame['Страна_Родители'] == 'Таджикистан', 'Страна_Родители'] = 'Республика Таджикистан'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Таджикистан Респ', 'Страна_Родители'] = 'Республика Таджикистан'

clean_frame.loc[clean_frame['Страна_Родители'] == 'РОССИЯ', 'Страна_Родители'] = 'Россия'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Росссия', 'Страна_Родители'] = 'Россия'
clean_frame.loc[clean_frame['Страна_Родители'] == 'Чеченская республика', 'Страна_Родители'] = 'Россия'

clean_frame.loc[clean_frame['Страна_Родители'] == 'КИТАЙ', 'Страна_Родители'] = 'Китай'
clean_frame.groupby('Страна_Родители')['ID'].count()


Страна_Родители
-                          200
Китай                       82
Кыргызская Республика       42
Монголия                     1
Республика Казахстан       239
Республика Таджикистан      46
Россия                    6076
Туркменистан                 2
Узбекистан                   3
Name: ID, dtype: int64

In [34]:
clean_frame.loc[clean_frame['Опекунство'].isnull()]
clean_frame.groupby('Опекунство')['ID'].count()

Опекунство
0.0    6685
1.0       6
Name: ID, dtype: int64

In [35]:

clean_frame.loc[(clean_frame['Село'].isnull()) & (~clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 1
clean_frame.loc[(clean_frame['Село'].isnull()) & (clean_frame['Город_ПП'].isin(['Барнаул', 'Барнаул г',])), 'Село'] = 0
clean_frame.loc[(clean_frame['Село'].isnull())]
clean_frame.groupby('Село')['ID'].count()

Село
0.0    5178
1.0    1513
Name: ID, dtype: int64

In [36]:

clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (~clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 1
clean_frame.loc[(clean_frame['Иностранец'].isnull()) & (clean_frame['Иностранец'].isin(['Россия'])), 'Иностранец'] = 0
clean_frame.loc[(clean_frame['Иностранец'].isnull())]
clean_frame.groupby('Иностранец')['ID'].count()

Иностранец
0.0    6095
1.0     596
Name: ID, dtype: int64

In [37]:
clean_frame.loc[(clean_frame['КодФакультета'].isnull())]
clean_frame.groupby('КодФакультета')['ID'].count()

КодФакультета
24.0     356
25.0    1060
26.0    1335
27.0     384
28.0     314
30.0     211
31.0     280
34.0     456
35.0     275
36.0     220
37.0       1
40.0     223
41.0     741
45.0      50
46.0      16
47.0      60
48.0     114
49.0      41
51.0     550
53.0       4
Name: ID, dtype: int64

In [38]:
clean_frame.loc[(clean_frame['КодФакультета'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,male,female,sex_summ,birth_year,entrance,birth_year_int,Статус


In [39]:
clean_frame.loc[(clean_frame['СрБаллАттестата'].isnull())]
big_atestat = clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'ID'].to_list()
small_atestat = clean_frame.loc[clean_frame['СрБаллАттестата'] < 5, 'ID'].to_list()
#clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'СрБаллАттестата'] = clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'СрБаллАттестата'] / 100
#clean_frame.loc[clean_frame['СрБаллАттестата'] > 100, 'СрБаллАттестата']
clean_frame.loc[clean_frame['ID'].isin(big_atestat), 'СрБаллАттестата']

614     5000.0
805     4118.0
1520     787.0
1651    1040.0
1686    2468.0
1789    4400.0
1988    4846.0
2154    3688.0
2698    4350.0
2729    4764.0
2812    4053.0
3191    4286.0
3399    3940.0
3505    2506.0
3788    5000.0
4073    7232.0
4239    3600.0
4528    3444.0
4889    4533.0
4966    3400.0
5342    4947.0
5355    5000.0
5432    4928.0
6503    3866.0
Name: СрБаллАттестата, dtype: float64

In [40]:
clean_frame.loc[(clean_frame['Статус'].isnull())]

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,Иностранец,КодФакультета,СрБаллАттестата,male,female,sex_summ,birth_year,entrance,birth_year_int,Статус


In [41]:
clean_frame.groupby('СрБаллАттестата')['ID'].count()

СрБаллАттестата
0.000       1
3.000       4
3.052       1
3.067       1
3.100       2
           ..
4846.000    1
4928.000    1
4947.000    1
5000.000    3
7232.000    1
Name: ID, Length: 343, dtype: int64

In [42]:
asu_frame.columns

Index(['ID', 'Код_группы', 'Год_Поступления', 'Пол', 'Основания',
       'Изучаемый_Язык', 'Дата_Рождения', 'Уч_Заведение', 'Где_Находится_УЗ',
       'Год_Окончания_УЗ', 'Пособие', 'Страна_ПП', 'Регион_ПП', 'Город_ПП',
       'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'male', 'female', 'sex_summ', 'birth_year', 'entrance',
       'birth_year_int', 'Статус'],
      dtype='object')

In [43]:
columns_to_csv = ['Год_Поступления', 'Пол', 'Основания', 'Изучаемый_Язык', 'Дата_Рождения', 'Страна_ПП', 'Общежитие', 'Наличие_Матери', 'Наличие_Отца', 'Страна_Родители',
       'Опекунство', 'Село', 'Иностранец', 'КодФакультета', 'СрБаллАттестата',
       'Статус', 'male', 'female', 'sex_summ', 'birth_year', 'entrance']

In [44]:
clean_frame.columns[clean_frame.isna().any()].tolist()

['Год_Окончания_УЗ']

In [45]:
clean_frame.loc[:, ['Основания', 'male', 'female']].head(10)

,Основания,male,female
0,ДН,0,1
1,ДН,0,1
2,ОО,0,1
3,ОО,0,1
4,ОО,0,1
5,ОО,0,1
6,СН,0,1
7,ОО,0,1
8,ДН,0,1
9,БН,0,1


# Составление справочников кодов для текстовых колонок

In [46]:
clean_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6691 entries, 0 to 6690
Data columns (total 30 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                6691 non-null   int64         
 1   Код_группы        6691 non-null   int64         
 2   Год_Поступления   6691 non-null   int64         
 3   Пол               6691 non-null   object        
 4   Основания         6691 non-null   object        
 5   Изучаемый_Язык    6691 non-null   object        
 6   Дата_Рождения     6691 non-null   object        
 7   Уч_Заведение      6691 non-null   object        
 8   Где_Находится_УЗ  6691 non-null   object        
 9   Год_Окончания_УЗ  6074 non-null   float64       
 10  Пособие           6691 non-null   float64       
 11  Страна_ПП         6691 non-null   object        
 12  Регион_ПП         6691 non-null   object        
 13  Город_ПП          6691 non-null   object        
 14  Общежитие         6691 n

In [47]:
# проверка полноты формирования спика колонок по форматам
non_digit_columns_num = clean_frame.dtypes[(clean_frame.dtypes == "datetime64[ns]") | (clean_frame.dtypes == "object")].count()
digit_columns_num = clean_frame.dtypes[(clean_frame.dtypes == "int64") | (clean_frame.dtypes == "float64")].count()
clean_frame.shape[1] - (non_digit_columns_num + digit_columns_num)

0

In [48]:
# список не числовых колонок
non_digit_columns = clean_frame.dtypes[(clean_frame.dtypes == "datetime64[ns]") | (clean_frame.dtypes == "object")].index.to_list()
non_digit_columns

['Пол',
 'Основания',
 'Изучаемый_Язык',
 'Дата_Рождения',
 'Уч_Заведение',
 'Где_Находится_УЗ',
 'Страна_ПП',
 'Регион_ПП',
 'Город_ПП',
 'Страна_Родители',
 'birth_year',
 'entrance']

In [49]:
# список числовых колонок

digit_columns = clean_frame.dtypes[(clean_frame.dtypes == "int64") | (clean_frame.dtypes == "float64")].index.to_list()
digit_columns

['ID',
 'Код_группы',
 'Год_Поступления',
 'Год_Окончания_УЗ',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'male',
 'female',
 'sex_summ',
 'birth_year_int',
 'Статус']

In [50]:
with open('data/codes.yaml', 'r') as file:
    codes_rule = yaml.load(file, Loader=yaml.FullLoader)
codes_rule

{'Город_ПП': {' ': 0,
  '-': 1,
  '10 лет Октября с': 2,
  '12 лет Октября п': 3,
  '3 Интернационал п': 4,
  'Абай': 5,
  'Абакан': 6,
  'Абакан г': 7,
  'Айхал п': 8,
  'Ак-Довурак': 9,
  'Акимовка с': 10,
  'Аксу': 11,
  'Аксу г': 12,
  'Акташ': 13,
  'Акташ с': 14,
  'Актобе': 15,
  'Акулово с': 16,
  'Алдан': 17,
  'Алейск': 18,
  'Алейск г': 19,
  'Алейский п': 20,
  'Александровка п': 21,
  'Александровка с': 22,
  'Алексеевка с': 23,
  'Аллак с': 24,
  'Алматы': 25,
  'Алматы г': 26,
  'Алмоси': 27,
  'Алтай': 28,
  'Алтай п': 29,
  'Алтайское': 30,
  'Алтайское с': 31,
  'Алтайское с.': 32,
  'Алхан-Кала с': 33,
  'Амур': 34,
  'Амур с': 35,
  'Анадырь': 36,
  'Андреевка п': 37,
  'Андроново с': 38,
  'Анжеро-Судженск': 39,
  'Анисимово с': 40,
  'Антоньевка с': 41,
  'Аньян г': 42,
  'Арбузовка с': 43,
  'Арбузовка с.': 44,
  'Армавир': 45,
  'Артемовский п': 46,
  'Артыбаш с': 47,
  'Ас с': 48,
  'Аскиз с': 49,
  'Астана': 50,
  'Астана г': 51,
  'Ауэзова п': 52,
  'Ахмет с'

In [51]:
columns_to_code = {
    'Основания': 'basis',
    'Изучаемый_Язык': 'language',    
    'Страна_ПП': 'country',
    #'Регион_ПП': 'region',
    #'Город_ПП': 'city',
    'Страна_Родители': 'parents_country',
}

In [52]:
list(columns_to_code.values())

['basis', 'language', 'country', 'parents_country']

In [53]:
columns_to_model = digit_columns + list(columns_to_code.values())
columns_to_model

['ID',
 'Код_группы',
 'Год_Поступления',
 'Год_Окончания_УЗ',
 'Пособие',
 'Общежитие',
 'Наличие_Матери',
 'Наличие_Отца',
 'Опекунство',
 'Село',
 'Иностранец',
 'КодФакультета',
 'СрБаллАттестата',
 'male',
 'female',
 'sex_summ',
 'birth_year_int',
 'Статус',
 'basis',
 'language',
 'country',
 'parents_country']

In [54]:
# Функция для преобразования слова в код по заданному правилу

def get_code(x: pd.Series, code_dict) -> pd.Series:    
    return code_dict[x]

In [55]:
clean_frame.groupby('Основания')['ID'].count()

Основания
БН     873
ДН     973
ОО    2565
СН    2177
ЦН     103
Name: ID, dtype: int64

In [56]:

for name, code in columns_to_code.items():     
    clean_frame[columns_to_code[name]] = clean_frame[name].apply(get_code, args=(codes_rule[name],))

clean_frame

,ID,Код_группы,Год_Поступления,Пол,Основания,Изучаемый_Язык,Дата_Рождения,Уч_Заведение,Где_Находится_УЗ,Год_Окончания_УЗ,...,female,sex_summ,birth_year,entrance,birth_year_int,Статус,basis,language,country,parents_country
0,64996,20608,2014,Жен,ДН,Английский язык,1995-11-03 00:00:00.000,"МБОУ ""СОШ №55""","Алтайский край, Барнаул г",2014.0,...,1,1,1995-11-03,2014-01-01,1995,0,1,1,8,10
1,71837,20613,2015,Жен,ДН,Немецкий язык,1995-12-10 00:00:00.000,"МБОУ ""СОШ №1""","Алтайский край, Рубцовск г",2014.0,...,1,1,1995-12-10,2015-01-01,1995,0,1,2,8,10
2,86587,21210,2018,Жен,ОО,Английский язык,1973-08-09 00:00:00.000,Алтайский государственный медицинский университет,"Алтайский край, г. Барнаул",1997.0,...,1,1,1973-08-09,2018-01-01,1973,0,2,1,8,10
3,73673,22254,2015,Жен,ОО,Английский язык,1983-12-07 00:00:00.000,"ГОУ ВПО ""Алтайский государственный университет""","Алтайский край, Барнаул г",2006.0,...,1,1,1983-12-07,2015-01-01,1983,0,2,1,8,10
4,54709,15040,2012,Жен,ОО,-,1994-05-21 00:00:00.000,-,-,NaN,...,1,1,1994-05-21,2012-01-01,1994,0,2,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6686,74342,20680,2016,Муж,БН,Английский язык,1992-03-06 00:00:00.000,ГУ Павлодарский профессиональный лицей №7,"Казахстан, Павлодарская обл, г Павлодар",2011.0,...,0,1,1992-03-06,2016-01-01,1992,0,0,1,6,7
6687,54876,16921,2013,Жен,ОО,-,1995-11-08 00:00:00.000,-,-,NaN,...,1,1,1995-11-08,2013-01-01,1995,0,2,0,8,10
6688,66879,19400,2014,Жен,СН,Английский язык,1996-03-31 00:00:00.000,"МБОУ ""Гимназия №42""","Алтайский край, Барнаул г",2014.0,...,1,1,1996-03-31,2014-01-01,1996,0,3,1,8,10
6689,64982,18152,2014,Муж,ОО,Немецкий язык,1996-08-22 00:00:00.000,"МБОУ ""Ребрихинская СОШ""","Алтайский край, Ребрихинский р-н, Ребриха ст",2014.0,...,0,1,1996-08-22,2014-01-01,1996,0,2,2,8,10


In [57]:
frame_to_csv = clean_frame.loc[clean_frame['СрБаллАттестата'] < 100]

In [60]:

frame_to_csv.loc[:, columns_to_model].to_csv('data/test_frame.csv', index_label='ind')

In [59]:
clean_frame.isna().any()

ID                  False
Код_группы          False
Год_Поступления     False
Пол                 False
Основания           False
Изучаемый_Язык      False
Дата_Рождения       False
Уч_Заведение        False
Где_Находится_УЗ    False
Год_Окончания_УЗ     True
Пособие             False
Страна_ПП           False
Регион_ПП           False
Город_ПП            False
Общежитие           False
Наличие_Матери      False
Наличие_Отца        False
Страна_Родители     False
Опекунство          False
Село                False
Иностранец          False
КодФакультета       False
СрБаллАттестата     False
male                False
female              False
sex_summ            False
birth_year          False
entrance            False
birth_year_int      False
Статус              False
basis               False
language            False
country             False
parents_country     False
dtype: bool